In [1]:
import keras
from keras.layers import LSTM
import numpy as np

Using TensorFlow backend.


In [2]:
from kp_data_loader import KPDataLoader

In [3]:
# Get data again just in case I messed up somewhere
loader = KPDataLoader()
for i in range(1,46):
    loader.load_file('ex{0}a.mid.csv'.format(i))
X, Y, m = loader.get_XX_and_YY()

In [4]:
from keras.preprocessing.sequence import pad_sequences
m

673

In [5]:
X_pad = pad_sequences(X, maxlen=m, padding='post')
Y_pad = pad_sequences(Y, maxlen=m, padding='post')

In [6]:
sample_weights = np.ones((45, m))
for i in xrange(len(X)):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0

In [7]:
from keras.models import Sequential

In [8]:
model = Sequential()

In [9]:
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers.wrappers import TimeDistributed

In [10]:
from keras.layers import Embedding

In [11]:
mask = np.zeros(12)
model.add(Masking(mask_value=mask, input_shape=(m, 12)))
model.add(LSTM(50, return_sequences=True, dropout_W=0.4, dropout_U=0.4))
model.add(TimeDistributed(Dense(12, activation="softmax")))

In [12]:
from keras.utils.np_utils import to_categorical

In [13]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              sample_weight_mode='temporal')

In [14]:
n = len(X)
j = int(n - (float(n) * 0.5))
X_train, X_test = X_pad[:j, :], X_pad[j:, :]

In [15]:
Y_train, Y_test = Y_pad[:j, :], Y_pad[j:, :]

In [16]:
sample_weights_train, sample_weights_test = sample_weights[:j, :], sample_weights[j:, :]

In [17]:
# # for custom metrics
import numpy as K

# def weighted_accuracy(y_true, y_pred):
#     score_array *= sample_weights
# score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))

def weighted_accuracy(y_true, y_pred):
    # Only for testing
#     score_array = K.equal(K.argmax(y_true, axis=-1),
#                           K.argmax(y_pred, axis=-1))
#     score_array *= weights
#     score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))
#     return K.mean(score_array)
    total = 0.0
    count = 0.0
    for i, y_i in enumerate(y_true):
        for j, y_ij in enumerate(y_i):
            if sum(y_ij) > 0:
                total += y_ij[y_pred[i][j]]
                count += 1
    return total / count

In [18]:
hist = model.fit(X_train, Y_train, batch_size=j, nb_epoch=500, sample_weight=sample_weights_train)

Epoch 1/500
22/22 [==============================] - 3s - loss: 11.4215
Epoch 2/500
22/22 [==============================] - 2s - loss: 11.2756
Epoch 3/500
22/22 [==============================] - 2s - loss: 11.1863
Epoch 4/500
22/22 [==============================] - 1s - loss: 11.1500
Epoch 5/500
22/22 [==============================] - 2s - loss: 11.0825
Epoch 6/500
22/22 [==============================] - 2s - loss: 10.9728
Epoch 7/500
22/22 [==============================] - 1s - loss: 11.0568
Epoch 8/500
22/22 [==============================] - 1s - loss: 10.9534
Epoch 9/500
22/22 [==============================] - 2s - loss: 11.0834
Epoch 10/500
22/22 [==============================] - 2s - loss: 10.9230
Epoch 11/500
22/22 [==============================] - 1s - loss: 10.8759
Epoch 12/500
22/22 [==============================] - 1s - loss: 10.7075
Epoch 13/500
22/22 [==============================] - 1s - loss: 10.8868
Epoch 14/500
22/22 [==============================] - 1s - l

In [19]:
Y_prediction = model.predict_classes(X_test, batch_size=5)

23/23 [==============================] - 1s     


In [20]:
# Y_prediction = to_categorical(Y_prediction)
# weighted_accuracy(Y_test, Y_prediction, sample_weights_test)
weighted_accuracy(Y_test, Y_prediction)

0.53969109195402298

In [21]:
model.evaluate(X_test, Y_test, batch_size=20, sample_weight=sample_weights_test)

23/23 [==============================] - 0s     


4.4423696061839228

In [22]:
Y_prediction[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  0,
        0,  0,  0,  9,  9,  9,  4,  7,  4,  4,  4,  4,  0,  4,  4,  7,  7,
        7,  0,  7,  0,  0,  0,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        0,  9,  5,  0,  5,  2,  7,  7,  7,  7,  0,  0,  0,  0,  0,  0,  9,
        9,  9,  9,  9,  9,  9,  9,  4,  4,  4,  4,  4,  4,  4,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  4,  4,  4,  4,  4,  4,  4,
        7,  0,  4,  7,  0,  0,  4,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        4,  4,  4,  4,  4,  4,  4,  4,  9,  9,  9,  9,  9,  9,  9, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11

In [23]:
loader = KPDataLoader()
loader.load_file('ex1a.mid.csv')
X, Y, m = loader.get_XX_and_YY()
X_pad = pad_sequences(X, maxlen=673, padding='post')
Y_pad = pad_sequences(Y, maxlen=673, padding='post')
sample_weights = np.ones((1, m))
for i in xrange(len(X)):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0
Y_prediction = model.predict_classes(X_pad, batch_size=1)

1/1 [==============================] - 0s


In [24]:
Y_prediction

array([[0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 9, 5, 5, 7, 2, 7, 0, 0, 0,
        0, 0, 7, 7, 7, 0, 7, 0, 0, 0, 0, 7, 0, 7, 7, 0, 0, 0, 0, 0, 0, 0,
        7, 7, 7, 0, 7, 7, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 9, 5, 5, 7,
        5, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 0, 7, 7, 0, 0, 0, 0, 7,
        0, 0, 0, 7, 0, 0, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [25]:
Y_pad[0][3]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
for w in model.get_weights():
    if len(w.shape) > 1:
        plt.figure()
        plt.imshow(w)

NameError: name 'plt' is not defined

In [ ]:
count = 0
for y in Y_prediction:
    if sum(y) > 0:
        count += 1
        print sum(y)
print count

In [ ]:
hist.history

In [ ]:
import cPickle
with open('test.pkl', 'w') as f:
    test = 100, hist
    cPickle.dump(test, f)

In [ ]:
with open('test.pkl', 'r') as f:
    test = cPickle.load(f)